<a href="https://www.kaggle.com/code/ashishamin/gdp-analysis?scriptVersionId=261977399" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv("/kaggle/input/gdp-per-country-20202025/2020-2025.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
null_df=df[df.isnull().any(axis=1)]
print(null_df)

* According to this, Eritrea and Syria needs to be dropped as we cannot move forward with these countries due to unavailability of the data.
* Other Countries null rows can be filled using Median value of the respective countries.

In [ ]:
df=df.drop([169,54])

 We dropped Syria and Eritrea using Index values. 

In [ ]:
num_cols=df.select_dtypes(include=["number"]).columns
df[num_cols]=df[num_cols].T.apply(lambda row:row.fillna(row.median())).T

Fill the rest of the countries' null values with the Median.

In [ ]:
df.isnull().sum()

In [ ]:
top20_2020 = df[['Country', '2020']].sort_values(by='2020', ascending=False).head(20)
plt.figure(figsize=(12,6))
plt.bar(top20_2020['Country'], top20_2020['2020'])
plt.xticks(rotation=75)
plt.title("Top 20 Countries by GDP in 2020")
plt.ylabel("GDP (current USD)")
plt.show()

In [ ]:
top20_2025 = df[['Country', '2025']].sort_values(by='2025', ascending=False).head(20)
plt.figure(figsize=(12,6))
plt.bar(top20_2025['Country'], top20_2025['2025'])
plt.xticks(rotation=75)
plt.title("Top 20 Countries by GDP in 2025")
plt.ylabel("GDP (current USD)")
plt.show()

In [ ]:
df['Change_2020_2025_%'] = ((df['2025'] - df['2020']) / df['2020']) * 100
df[['Country', '2020', '2025', 'Change_2020_2025_%']].sort_values(by='Change_2020_2025_%', ascending=False).head(10)

top10_growth = df.sort_values(by='Change_2020_2025_%', ascending=False).head(10)
plt.figure(figsize=(12,6))
plt.bar(top10_growth['Country'], top10_growth['Change_2020_2025_%'])
plt.xticks(rotation=75)
plt.title("Top 10 Countries by GDP Growth (2020–2025)")
plt.ylabel("Growth (%)")
plt.show()

2026 GDP Prediction using Linear Regression
We apply a linear regression model for each country (2020–2025) to estimate GDP values for 2026.

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
years = np.array([2020, 2021, 2022, 2023, 2024, 2025]).reshape(-1, 1)
df['2026_pred'] = np.nan
for i, row in df.iterrows():
    values = row[['2020', '2021', '2022', '2023', '2024', '2025']].values.astype(float)
    
    model = LinearRegression()
    model.fit(years, values)

    pred_2026 = model.predict(np.array([[2026]]))[0]
    df.at[i, '2026_pred'] = pred_2026
df[['Country', '2025', '2026_pred']].head(10)

In [ ]:
india = df[df['Country'] == 'India'].iloc[0]

years = [2020, 2021, 2022, 2023, 2024, 2025]
values = [india[str(y)] for y in years]

gdp_2026 = india['2026_pred']

years_extended = years + [2026]
values_extended = values + [gdp_2026]


plt.figure(figsize=(8,5))
plt.plot(years_extended, values_extended, marker='o', label="GDP")
plt.axvline(2025.5, color='red', linestyle='--', label="2026 Prediction")
plt.title("India GDP Trend (2020–2026, with Prediction)")
plt.xlabel("Year")
plt.ylabel("GDP (current USD)")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
world_gdp = df[['2020','2021','2022','2023','2024','2025','2026_pred']].sum()

years = world_gdp.index.tolist()
values = world_gdp.values

plt.figure(figsize=(8,5))
plt.plot(years, values, marker='o')
plt.title("World GDP Trend (2020–2026, including prediction)")
plt.xlabel("Year")
plt.ylabel("Total GDP (current USD)")
plt.grid(True)
plt.show()